In [1]:
!pip install tensorflow

In [2]:
!pip install scikeras

In [76]:
import pandas as pd
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from scikeras.wrappers import KerasClassifier
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
df = pd.read_csv("data/points_data.csv")

In [3]:
# Select quantitatives variables
cuant_var = df[["altitud", "pendiente", "orientacion", "t_max", "u", "v", "specific_humidity", "relative_humidity"]]
# Normalize
mean = cuant_var.mean()
mean.to_csv("data/means.csv")
std = cuant_var.std()
std.to_csv("data/stds.csv")
normalized_df = (cuant_var-cuant_var.mean())/cuant_var.std()

In [4]:
# Split dataset
y = df["is_fire"]
xTrain, xTest, yTrain, yTest = train_test_split(normalized_df, y, test_size=0.2, random_state=12)

In [18]:
xTest

,altitud,pendiente,orientacion,t_max,u,v,specific_humidity,relative_humidity
518,-0.293634,0.165739,1.406116,-0.119670,1.500631,0.522767,0.107386,0.694872
871,-0.954662,-0.827133,0.010550,-1.204439,0.311809,-0.834331,-0.903463,-0.018830
797,-0.576325,-0.832027,-0.043692,-1.091953,-0.654979,-0.306512,-0.398518,0.588766
274,-0.956474,-0.881371,0.507296,-1.250444,0.438949,-2.396988,-0.962472,-0.228474
325,-0.031658,1.800683,-0.880576,0.757039,-0.164712,-0.664707,1.873592,1.023697
...,...,...,...,...,...,...,...,...
909,-0.588225,-0.618137,-0.037166,0.164613,-1.442333,0.058375,0.037396,-0.091951
757,0.236402,0.646659,-0.365338,-0.885517,0.714322,-0.153783,-0.388918,0.378511
965,2.912808,-0.613062,0.582406,0.658129,-0.048901,-0.200054,0.599300,-0.184310
4,-0.905743,-0.838445,-1.302362,-0.413200,1.866672,0.078302,-0.088814,0.455880


In [16]:
yTest

518    False
871    False
797    False
274    False
325    False
       ...  
909    False
757    False
965    False
4      False
151    False
Name: is_fire, Length: 200, dtype: bool

In [7]:
# Create function returning a model
def design_model (dropout_rate, weight_constraint):

    model = keras.Sequential()

    model.add(keras.layers.Dense(64, input_shape=(8,), activation='relu', kernel_constraint=keras.constraints.MaxNorm(weight_constraint)))
    model.add(keras.layers.Dropout(dropout_rate))

    model.add(keras.layers.Dense(128, activation='relu', kernel_constraint=keras.constraints.MaxNorm(weight_constraint)))
    model.add(keras.layers.Dropout(dropout_rate))

    model.add(keras.layers.Dense(256, activation='relu', kernel_constraint=keras.constraints.MaxNorm(weight_constraint)))
    model.add(keras.layers.Dropout(dropout_rate))

    model.add(keras.layers.Dense(128, activation='relu', kernel_constraint=keras.constraints.MaxNorm(weight_constraint)))
    model.add(keras.layers.Dropout(dropout_rate))

    model.add(keras.layers.Dense(64, activation='relu', kernel_constraint=keras.constraints.MaxNorm(weight_constraint)))
    model.add(keras.layers.Dropout(dropout_rate))
    
    model.add(keras.layers.Dense(2, activation='softmax'))
    
    model.compile(loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    return model

model = KerasClassifier(model=design_model, verbose=0)

In [8]:
# Define the search parameters
epochs = [10,20,50,100]
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
weight_constraint = [1.0, 2.0, 3.0, 4.0, 5.0]
dropout_rate = [ 0.2, 0.3, 0.4, 0.5]

In [10]:
param_grid = dict(model__dropout_rate=dropout_rate, model__weight_constraint=weight_constraint, optimizer=optimizer, epochs=epochs)

# grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

In [11]:
grid_result = grid.fit(xTrain, yTrain)

PicklingError: Could not pickle the task to send it to the workers.

In [ ]:
# Print results
print(f'Best:{grid_result.best_score_} using {grid_result.best_params_}')

for mean, stdev, param in zip(grid_result.cv_results_['mean_test_score'], 
                              grid_result.cv_results_['std_test_score'], grid_result.cv_results_['params']):
    print(f"{mean} {stdev} with: {param}")

In [ ]:
##### Design final model with best parameters
model = keras.Sequential()

model.add(keras.layers.Dense(64, input_shape=(19,), activation='relu', kernel_constraint=keras.constraints.MaxNorm(1)))
model.add(keras.layers.Dropout(0.3))

model.add(keras.layers.Dense(128, activation='relu', kernel_constraint=keras.constraints.MaxNorm(1)))
model.add(keras.layers.Dropout(0.3))

model.add(keras.layers.Dense(256, activation='relu', kernel_constraint=keras.constraints.MaxNorm(1)))
model.add(keras.layers.Dropout(0.3))

model.add(keras.layers.Dense(128, activation='relu', kernel_constraint=keras.constraints.MaxNorm(1)))
model.add(keras.layers.Dropout(0.3))

model.add(keras.layers.Dense(64, activation='relu', kernel_constraint=keras.constraints.MaxNorm(1)))
model.add(keras.layers.Dropout(0.3))
    
model.add(keras.layers.Dense(2, activation='softmax'))

In [ ]:
# Define the accuracy metrics and parameters
model.compile(optimizer="Adamax", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Run the model
model.fit(xTrain, yTrain, epochs=50)

In [ ]:
# Export model
model.save("data/nn_fire_risk.h5")

In [6]:
# Load model
model = keras.models.load_model('data/nn_fire_risk.h5')

In [7]:
# Predict for test data 
yTestPredicted = model.predict(xTest)
yTestPredicted = yTestPredicted[:,1]

# Calculate the error metrics
yTestPredicted = (yTestPredicted>0.5).astype(int)
cMatrix = confusion_matrix(yTest, yTestPredicted)
pScore = precision_score(yTest, yTestPredicted)
rScore = recall_score(yTest, yTestPredicted)

print("\nConfusion matrix:", cMatrix)
print("\nP-Score: %.3f, R-Score: %.3f" % (pScore, rScore))

7/7 [==============================] - 0s 661us/step

Confusion matrix: [[168   2]
 [  6  24]]

P-Score: 0.923, R-Score: 0.800


In [9]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(xTest, yTest)

# Print the evaluation results
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

7/7 [==============================] - 0s 922us/step - loss: 0.1359 - accuracy: 0.9600
Test Loss: 0.1358533501625061
Test Accuracy: 0.9599999785423279


In [14]:
!pip install tensorflowjs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.6/100.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.9/189.9 kB 3.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 4.1 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 2.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.7/238.7 kB 5.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.6/164.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.9/68.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.0/125.0 kB 5.0 MB/s eta 0:00:00
  Installing build depen

In [ ]:
import tensorflowjs as tfjs

# Convert the Keras model to TensorFlow.js format
tfjs.converters.save_keras_model(model, '/data/nn_fire_risk_js')

In [26]:
xTest_list_1 = [546.37,21.421358,80.22286,300.30322265625,4.351698398590088,6.368526458740234,0.005158774554729462,27.214874267578125]
xTest_1 = pd.DataFrame([xTest_list_1])

In [27]:
type(xTest_1)

pandas.core.frame.DataFrame

In [28]:
xTest_1

,0,1,2,3,4,5,6,7
0,546.37,21.421358,80.22286,300.303223,4.351698,6.368526,0.005159,27.214874


In [20]:
type(xTest)

pandas.core.frame.DataFrame

In [33]:
xTest_list_1 = [546.37,21.421358,80.22286,300.30322265625,4.351698398590088,6.368526458740234,0.005158774554729462,27.214874267578125]
xTest_1 = pd.DataFrame([xTest_list_1])
yTest_1_predicted = model.predict(xTest_1)
yTest_1_predicted

1/1 [==============================] - 0s 16ms/step


array([[0., 1.]], dtype=float32)

In [37]:
xTest_1

,0,1,2,3,4,5,6,7
0,546.37,21.421358,80.22286,300.303223,4.351698,6.368526,0.005159,27.214874


In [35]:
yTest_1_predicted

array([[0., 1.]], dtype=float32)

In [34]:
xTest_list_2 = [10.5720005,2.4017253,330.45306,295.8662109375,-0.29025888442993164,-4.314094543457031,0.011048879474401474,65.92393493652344]
xTest_2 = pd.DataFrame([xTest_list_2])
yTest_2_predicted = model.predict(xTest_2)
yTest_2_predicted

1/1 [==============================] - 0s 12ms/step


array([[0., 1.]], dtype=float32)

In [40]:
xTest_list_3 = [xTest_list_1, xTest_list_2]
xTest_list_3

[[546.37,
  21.421358,
  80.22286,
  300.30322265625,
  4.351698398590088,
  6.368526458740234,
  0.005158774554729462,
  27.214874267578125],
 [10.5720005,
  2.4017253,
  330.45306,
  295.8662109375,
  -0.29025888442993164,
  -4.314094543457031,
  0.011048879474401474,
  65.92393493652344]]

In [42]:
xTest_3 = pd.DataFrame(xTest_list_3)
xTest_3

,0,1,2,3,4,5,6,7
0,546.370,21.421358,80.22286,300.303223,4.351698,6.368526,0.005159,27.214874
1,10.572,2.401725,330.45306,295.866211,-0.290259,-4.314095,0.011049,65.923935


In [46]:
xTest_3 = xTest.copy()

In [49]:
xTest_3.iloc[0]

altitud             -0.293634
pendiente            0.165739
orientacion          1.406116
t_max               -0.119670
u                    1.500631
v                    0.522767
specific_humidity    0.107386
relative_humidity    0.694872
Name: 518, dtype: float64

In [56]:
xTest_3.head(3)

,altitud,pendiente,orientacion,t_max,u,v,specific_humidity,relative_humidity
518,-0.293634,0.165739,1.406116,-0.119670,1.500631,0.522767,0.107386,0.694872
871,-0.954662,-0.827133,0.010550,-1.204439,0.311809,-0.834331,-0.903463,-0.018830
797,-0.576325,-0.832027,-0.043692,-1.091953,-0.654979,-0.306512,-0.398518,0.588766


In [69]:
xTest_3 = xTest.head(100)

In [70]:
xTest_3

,altitud,pendiente,orientacion,t_max,u,v,specific_humidity,relative_humidity
518,-0.293634,0.165739,1.406116,-0.119670,1.500631,0.522767,0.107386,0.694872
871,-0.954662,-0.827133,0.010550,-1.204439,0.311809,-0.834331,-0.903463,-0.018830
797,-0.576325,-0.832027,-0.043692,-1.091953,-0.654979,-0.306512,-0.398518,0.588766
274,-0.956474,-0.881371,0.507296,-1.250444,0.438949,-2.396988,-0.962472,-0.228474
325,-0.031658,1.800683,-0.880576,0.757039,-0.164712,-0.664707,1.873592,1.023697
...,...,...,...,...,...,...,...,...
403,1.438310,1.671688,-0.820983,1.381804,0.563844,1.368574,-0.874234,-1.824139
886,-0.938628,-0.870692,-0.881941,-0.174605,0.231181,-0.121332,-0.753221,-1.031635
542,-0.956001,-0.850536,0.793985,-0.494520,-0.974360,1.239939,0.257952,1.031888
29,-0.778937,-0.648503,0.135234,0.407176,-0.462611,-0.320644,1.506346,1.109932


In [73]:
yTest_3_predicted = model.predict(xTest_3)
results_3 = (yTest_3_predicted[:,1]>0.5).astype(int)

4/4 [==============================] - 0s 1ms/step


In [74]:
results_3

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0])

In [77]:
np.where(results_3 == 1)

(array([27, 35, 45, 48, 71, 76, 81, 83, 86, 87, 88, 90, 91, 95]),)

In [80]:
xTest_3.iloc[27]

altitud              1.068535
pendiente            0.179948
orientacion          0.354402
t_max                1.389336
u                    0.554170
v                    1.370858
specific_humidity   -0.890492
relative_humidity   -1.837286
Name: 488, dtype: float64